In [8]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt
from collections import ChainMap
from util import *

from xgboost.sklearn import XGBClassifier

In [2]:
x_train, x_test, x_oot, y_train, y_test, y_oot = load_data('var_30.csv')
x_train.head()

,ssn_name_count_14,ssn_name_dob_count_14,ssn_lastname_count_14,ssn_firstname_count_14,ssn_dob_count_0_by_30,name_dob_count_0_by_30,fulladdress_homephone_count_14,fulladdress_count_1_by_14,fulladdress_homephone_day_since,ssn_dob_day_since,...,fulladdress_count_0,name_dob_count_3,fulladdress_homephone_count_0_by_14,name_day_since,name_count_14,ssn_dob_count_0_by_14,ssn_lastname_count_0_by_14,ssn_firstname_count_0_by_14,ssn_name_day_since,address_count_1_by_7
644385,-0.070962,-0.068281,-0.071051,-0.071088,0.162987,0.163303,-0.070089,0.123861,0.664103,0.656572,...,-0.041351,-0.046415,0.120387,0.874335,-0.159376,0.118004,0.123458,0.123528,0.675430,0.103441
445533,-0.070962,-0.068281,-0.071051,-0.071088,0.162987,0.163303,-0.070089,0.123861,-0.019436,-0.026541,...,-0.041351,-0.046415,0.120387,0.193700,-0.159376,0.118004,0.123458,0.123528,-0.009232,0.103441
742738,-0.070962,-0.068281,-0.071051,-0.071088,0.162987,0.163303,-0.070089,0.123861,1.005872,0.998129,...,-0.041351,-0.046415,0.120387,1.214653,-0.159376,0.118004,0.123458,0.123528,1.017761,0.103441
337890,-0.070962,-0.068281,-0.071051,-0.071088,0.162987,0.163303,-0.070089,0.123861,-0.389686,-0.396561,...,-0.041351,-0.046415,0.120387,-0.174977,-0.159376,0.118004,0.123458,0.123528,-0.380091,0.103441
234259,-0.070962,-0.068281,-0.071051,-0.071088,0.162987,0.163303,-0.070089,0.123861,-0.750443,-0.757094,...,-0.041351,-0.046415,0.120387,-0.534202,-0.159376,0.118004,0.123458,0.123528,-0.741440,0.103441


In [ ]:
bt = XGBClassifier(learning_rate=0.05, subsample=0.5, n_jobs=2)
params = {'n_estimators': [50, 100, 200, 400],
          'max_depth': [2, 5, 10, 20]}
grid = GridSearchCV(bt, params, cv=4, refit='FDR', 
                    verbose=1, n_jobs=4,
                    scoring={'FDR': fdr})
grid.fit(x_train, y_train)

Fitting 4 folds for each of 16 candidates, totalling 64 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed: 16.5min


In [16]:
print(f"The best parameters are {grid.best_params_} "
      f"with a score of {grid.best_score_:0.5f}")
pd.DataFrame(grid.cv_results_).filter(
    items=['param_max_depth', 'param_n_estimators', 
           'mean_test_FDR', 'std_test_FDR', 'rank_test_FDR',
           'mean_test_ROC', 'std_test_ROC'])

The best parameters are {'max_depth': 5, 'n_estimators': 200} with a score of 0.52772


,param_max_depth,param_n_estimators,mean_test_FDR,std_test_FDR,rank_test_FDR
0,2,50,0.485094,0.010583,16
1,2,100,0.506625,0.009317,14
2,2,200,0.515342,0.008095,13
3,2,400,0.521444,0.010321,9
4,5,50,0.500087,0.006975,15
5,5,100,0.518044,0.008510,11
6,5,200,0.527720,0.009671,1
7,5,400,0.527022,0.010692,2
8,10,50,0.517608,0.007837,12
9,10,100,0.527022,0.009708,2


In [17]:
bt = XGBClassifier(n_estimators=200,
                   max_depth=5,
                   learning_rate=0.05,
                   subsample=0.5,
                   verbose=3, n_jobs=8).fit(x_train, y_train)
print(fdr(bt, x_train, y_train))
print(fdr(bt, x_test, y_test))

0.5328626220362622
0.5302978432043821
